#COMPSCI 546: Applied Information Retrieval 
##Assignment 4: Relevance Models (Total : 100 points)



# Download input files and code

Please execute the cell below to download the input files. 

In [ ]:

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


import os
import zipfile

download = drive.CreateFile({'id': '1CLVPQ0HADjmE0cPthdZ628aIYc8yvMdq'})
download.GetContentFile('HW04.zip')

with zipfile.ZipFile('HW04.zip', 'r') as zip_file:
    zip_file.extractall('./')
os.remove('HW04.zip')
# We will use hw1 as our working directory
os.chdir('HW04')

#Setting the input files 
queries_file = "queries_tok_clean_kstem"
col = "antique-collection.tok.clean_kstem"
query_pass = "query_pass"    

# 1 : Initial Data Setup (20 points)

We use files from the ANTIQUE  [https://arxiv.org/pdf/1905.08957.pdf] dataset for this assignment. As described in the previous assignments, this is a passage retrieval dataset. 

The description of the input files provided for this assignment is given below.

**Query File**

We randomly sampled a set of 5 queries from the test set of the ANTIQUE dataset. Each row of the input file contains the following information:

*query_id query_text*

The id and text information is tab separated. query_id is a unique identifier for a query and query text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer.  

**Collection file**

Each row of the file consists of the following information:

*passage_id  passage_text*

The id and text information is tab separated. The passage text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer (same as queries). The terms in the passage text can be accessed by splitting the text based on space.

**Query Feedback Passages**

This file consists of queries and 10 feedback passages corresponding to each query.  Each row of the file consists of the following information:

*query_id  passage_id1 passage_id2 ......passage_id10*

The entries are space separated. The first column is the query_id followed by the passage_ids. 

In this section, you have to implement the following: 

* Load the queries from the query file into a datastructure
* Load the query feedback passages into a datastructure.

You can use any additional datastructures than the suggested ones for your implementation. 




In [ ]:
from collections import defaultdict
''' 
This function is used to load query file information into datastructure(s).
Return Variables: 
queries - mapping from queryid to querytext
'''

def loadQueries(queries_file):
  #enter your code here 
  queries = defaultdict()
  file = open(queries_file, "r")
  for q in file:
    temp = q.split()
    queries[temp[0]] = temp[1:]
  file.close()
  return queries 


'''
This function is used to load feedback passages corresponding to the queries into a datastructure.
The file format is given below:
"query_id passage_id1 passage_id2 .....   passage_id10"
The entries are space separated. 

Return Variables:
num_queries - number of queries in the file
feedback_pass - mapping from queryid to list of feedback passages                
'''


def loadFeedbackPass(query_pass):
     #enter your code here
     feedback_pass = {}
     file = open(query_pass, "r")
     for q in file:
        temp = q.split()
        feedback_pass[temp[0]] = temp[1:]
     file.close()
     return feedback_pass



queries = loadQueries(queries_file)
feedback_pass = loadFeedbackPass(query_pass)


print ('Total Num of queries in the query file  : {0}'.format(len(queries)))
print ('Total Num of queries in the feedback file  : {0}'.format(len(feedback_pass)))



Total Num of queries in the query file  : 5
Total Num of queries in the feedback file  : 5



In the cell below, an inverted index with count has been created in memory. Please run the cell and use the variables for implementing the relevance feedback models. 

In [ ]:
'''
Store the feedback docids
'''

def storeFeedbackPass(query_pass):
    feedback_pass_ids = {}
    for line in open(query_pass):
      row = line.strip().split(' ')      
      for p in row[1:]:
        if p not in feedback_pass_ids:
          feedback_pass_ids[p]=0
        feedback_pass_ids[p]+=1
    return feedback_pass_ids

feedback_pass_ids = storeFeedbackPass(query_pass)


'''
An inverted index with count information. 
'''
class indexCount:
   pcount = 0 
   ctf = {}
   sumdl = 0
   avgdl = 0  
   doclen = {}
   index = {}
   probctf = {}
   feedback_pass_contents = {}
   
   def __init__(self, col):
     self.col = col
     
   
   def create_index(self):
     for line in open(self.col):    
       pid,ptext = line.strip().split('\t')
       self.pcount+=1 
       
       if pid not in self.doclen:
           self.doclen[pid]=0
       pfreq = {}
       for term in ptext.split(' '):
           self.doclen[pid]+=1 
           
           if term not in pfreq:
              pfreq[term]=0
           pfreq[term]+=1
       
       
       for k,v in pfreq.items():
        if k not in self.index:
          self.index[k]={}      
        
        self.index[k][pid]=v 

       if pid in feedback_pass_ids:
          self.feedback_pass_contents[pid]=ptext
       
     
buildIndex = indexCount(col)      
buildIndex.create_index()


''' 
inverted index with count: nested dict with term and passage_id as key"
Example - {'the': {'2020338_0:11', '3174498_1:4'}} 
'''  
index = buildIndex.index

#total number of passages in the collection 
num_passages = buildIndex.pcount

#dict with passageId as key and number of terms in the passage as value  
doclen = buildIndex.doclen

#dict with passage id as key and passage text as value for feedback passages
feedback_pass_contents = buildIndex.feedback_pass_contents

print('Total number of passages in the collection :{0}'.format(num_passages))
print('Total number of feedback passages :{0}'.format(len(feedback_pass_contents)))


Total number of passages in the collection :403492
Total number of feedback passages :50


# 2 : Query Language Model (20 points)

In the cell below, calculate Maximum Likelihood Estimates for the Query Language Model, as follows:

$P_{MLE}(t|q)$ = $\frac{count(t,q)}{|q|}$ 

$count(t,q)$ - Number of times term $t$ appears in query $q$

$|q|$ - Number of tokens in query $q$

Calculate the estimates for all queries.


In [ ]:
''' 
In this function implement mle estimates for all queries.
Return Variables: 
  mle_queries - mapping from queryid to mle estimates. The mle estimates for each query is a dict from each query word to its MLE probability.
'''

def calcMleQueries(queries):
  #enter your code here
  mle_queries = defaultdict(dict)
  for qid, qterm in queries.items():
    fmap = defaultdict(int)
    nterm = len(qterm)
    for q in qterm:
      fmap[q] += 1

    for q in qterm:
      mle_queries[qid][q] = fmap[q]/nterm

  return mle_queries 

mle_queries = calcMleQueries(queries)  

# Hint: the MLE estimates are in the interval [0.10,0.20]
print('MLE estimates for qid 3396066 :{0}'.format(mle_queries['3396066']))


MLE estimates for qid 3396066 :{'why': 0.16666666666666666, 'do': 0.16666666666666666, 'airplane': 0.16666666666666666, 'fly': 0.16666666666666666, 'so': 0.16666666666666666, 'high': 0.16666666666666666}


# 3 : Relevance Model 1 (RM1) (30 points)

In the cell below, implement the RM1 feedback model, as follows. 

$$P_{RM1}(t|\theta_{F}) \propto \sum_{p \in F} (p(t|\theta_{p}) \prod_{w \in q} p(w|\theta_{p}))$$

$p(t|\theta_{p}) = \frac{count(t,p) + \delta}{|p| + \delta|V|}$ ($p(w|\theta_{p})$ is computed similarly)

$\delta = 0.1$

$|V|$: vocabulary size (number of unique words in the vocabulary)

$count(t,p)$ - Number of times term $t$ occurs in passage $p$

$|p|$ - Number of tokens in passage $p$

$F$: Feedback passages

For every query, this has to be computed for all unique terms present in feedback passages.

**Note:** Once you compute the weights for each term, you should normalize them by dividing by their sum. In other words, the RM1 probabilities for each query should sum to 1.


In [ ]:
import collections

''' 
In this function implement RM1 and return words and their probabilities.
Return Variables: 
rm1 - mapping from queryid to RM1 probabilities. 
The RM1 probabilities for each query is a dict from the words that appear in the feedback passages to their RM1 probability. 
'''

def calcRM1(index,queries,doclen,feedback_pass,feedback_pass_contents):   
    #enter your code here 
    rm1 = defaultdict(dict)
    for qid, qterm in queries.items():
      uniq = defaultdict(int)
      pid = feedback_pass[qid]
      prob = 0
      
      for p in pid:
        psplit = feedback_pass_contents[p].split(" ")
        for term in psplit:
          uniq[term] += 1
      
      for t, val in uniq.items():
        pprob = 0
        for p in pid:
          qprob = 1
          tp_count = 0
          allterms = doclen[p]
          
          for qi in qterm:
            wp_count = 0
            if p in index[qi]:
              wp_count = index[qi][p]

            qprob *= ((wp_count + 0.1)/(allterms + (0.1 * len(index))))
                     
          if p in index[t]:
            tp_count = index[t][p]    

          pprob += (((tp_count + 0.1)/(allterms + (0.1 * len(index)))) * qprob)
        
        rm1[qid][t] = pprob
        prob += pprob

    for k, v in rm1[qid].items():
      rm1[qid][k] = v/prob
                  
    return rm1 

rm1 = calcRM1(index,queries,doclen,feedback_pass,feedback_pass_contents)

'''
Print out top 20 terms and corresponding probabilities
this assumes that the rm1 variable returned is a dict with queryid as key and dict consisting of term and probability values as the value.
You can alter this based on your implementation.
'''

rm1_scores = {} 
rm1_final = {}
for k,v in rm1.items():
    if k not in rm1_final:
      rm1_final[k]=[]
    if k not in rm1_scores:
      rm1_scores[k]={}  
    sorted_p = sorted(v.items(), key=lambda x: x[1], reverse=True)
    sorted_dict = collections.OrderedDict(sorted_p)
    for t,s in sorted_dict.items():
        rm1_final[k].append(t+":"+str(s)) 
        rm1_scores[k][t]=s

''' Hint: The probability value for the top (1st) term is in the range [0.081,0.083]
          The probability value for the 20th term is in the range [0.0076,0.0078]'''
print('Top 20 Feedback terms and their RM1 probabilities for qid 3396066 :{0}'.format(rm1_final['3396066'][:20]))



Top 20 Feedback terms and their RM1 probabilities for qid 3396066 :['the:0.08201915132041583', 'to:0.03889613136203955', 'a:0.038125031057436296', 'helicopter:0.030245917173091285', 'of:0.025662950681680185', 'and:0.024377340618260992', 'in:0.017980664332681986', 'can:0.016712341975101054', 'you:0.011984003910296985', 'that:0.011835938393553782', 'wings:0.011446867894048711', 'rotor:0.011434260221575728', 'are:0.010587661560101626', 'on:0.010233185208287796', 'control:0.010180149758141356', 'spin:0.010180149758141356', 'is:0.009495177908547764', 'fly:0.008124808252820954', 's:0.007781753950717449', 'engine:0.007672588083067976']


# 4 : Relevance Model 3 (RM3) (30 points)

In the cell below, implement RM3 feedback model

$$P_{RM3}(t|\theta_{F}) = \alpha P_{RM1}(t|\theta_{F}) + (1-\alpha) P_{MLE}(t|q)$$

$P_{RM1}(t|\theta_{F})$ - Probability score returned by RM1 model in Question 3

$P_{MLE}(t|q)$ - Query Likelihood estimate calculated in Question 2 

$\alpha = 0.5$
 
For every query, this has to be computed for all unique terms present in feedback passages and the query text.

In [ ]:
''' 
In this function implement RM1 and return words and their probabilities.
Return Variables: 
rm3 - mapping from queryid to RM1 probabilities. The RM3 probabilities for each query is a dict from the words that appear in the feedback passages or the query to their RM3 probability. 
'''
import collections

def calcRM3(rm1_scores,mle_queries,queries,doclen,feedback_pass,feedback_pass_contents):
    #enter your code here 
    rm3 = defaultdict(dict)
    for qid, qtext in queries.items():
      uniq = defaultdict(int)
      uniq_pid = feedback_pass[qid]

      for p in uniq_pid:
        pfb = feedback_pass_contents[p].split(" ")
        for i in pfb:
          uniq[i] += 1

      for qterm in qtext:
        uniq[qterm] += 1

      for uk in uniq.keys():
        p1 = 0
        p2 = 0
        if uk in rm1_scores[qid]:
          p1 = 0.5 * rm1_scores[qid][uk]
        if uk in mle_queries[qid]:
          p2 = (1 - 0.5) * mle_queries[qid][uk]

        rm3[qid][uk] = p1 + p2

    return rm3 


rm3 = calcRM3(rm1_scores,mle_queries,queries,doclen,feedback_pass,feedback_pass_contents)

'''
Print out top 20 terms and corresponding probabilities
this assumes that the rm3 variable returned is a dict with queryid as key and dict consisting of term and probability values as the value.
You can alter this based on your implementation.
'''

rm3_final = {}
for k,v in rm3.items():
    if k not in rm3_final:
      rm3_final[k]=[]
    sorted_p = sorted(v.items(), key=lambda x: x[1], reverse=True)
    sorted_dict = collections.OrderedDict(sorted_p)
    for t,s in sorted_dict.items():
        rm3_final[k].append(t+":"+str(s)) 
        
''' Hint: The probability value for the top (1st) term is in the range [0.087,0.088]
          The probability value for the 20th term is in the range [0.0050,0.0052]'''
print('Top 20 Feedback terms and their RM3 probabilities for qid 3396066 :{0}'.format(rm3_final['3396066'][:20]))



Top 20 Feedback terms and their RM3 probabilities for qid 3396066 :['fly:0.0873957374597438', 'so:0.08623512280241481', 'airplane:0.08608183924437131', 'do:0.08538803626235386', 'high:0.08415062068953452', 'why:0.08350777780768794', 'the:0.041009575660207916', 'to:0.019448065681019776', 'a:0.019062515528718148', 'helicopter:0.015122958586545642', 'of:0.012831475340840092', 'and:0.012188670309130496', 'in:0.008990332166340993', 'can:0.008356170987550527', 'you:0.005992001955148493', 'that:0.005917969196776891', 'wings:0.005723433947024356', 'rotor:0.005717130110787864', 'are:0.005293830780050813', 'on:0.005116592604143898']
